In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_parquet("Shopping Carts.parquet")
df.reset_index(inplace=True)

In [4]:
df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index()
df = pd.merge(df,df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index())

### Welche Counties sind sich ähnlich in Hinblick auf die jeweiligen „Top 10“-Produkte?

In [5]:
df_3 = df.groupby(["county","product_name"]).agg(anzahl=("product_name","count")).reset_index()

In [132]:
df_3[df_3.county == "San Bernardino"]


,county,product_name,anzahl,rank
389555,San Bernardino,100% Colombian Medium Dark Roast Ground Coffee,2,15.0
389556,San Bernardino,Almond Breeze Original Almond Milk,1,31.0
389557,San Bernardino,April Fresh Liquid Fabric Softener,4,2.0
389558,San Bernardino,Backyard Barbeque Potato Chips,2,15.0
389559,San Bernardino,Bag of Organic Bananas,1,31.0
...,...,...,...,...
389643,San Bernardino,Versatile Stain Remover 156 Loads,1,31.0
389644,San Bernardino,Vodka Pasta Sauce,1,31.0
389645,San Bernardino,White Antibacterial Deodorant Soap Bars,4,2.0
389646,San Bernardino,White Sliced Mushrooms,1,31.0


In [6]:
counties = df_3.county.unique().tolist()
for i in counties:
    df_3.loc[df_3["county"] == i,"rank"] = df_3.loc[df_3["county"] == i].anzahl.rank(method="min",ascending=False)

In [7]:
df_4 = df_3[df_3["rank"].isin(range(11))]
df_4 = df_4.sort_values(["county","rank"])

In [12]:
df_4

,county,product_name,anzahl,rank
233,Alameda,Banana,352,1.0
218,Alameda,Bag of Organic Bananas,269,2.0
654,Alameda,Clementines,177,3.0
2351,Alameda,Organic Strawberries,168,4.0
1219,Alameda,Granny Smith Apples,139,5.0
...,...,...,...,...
576879,Yuba,Sparkling Water Grapefruit,668,6.0
573171,Yuba,Organic Half & Half,653,7.0
577190,Yuba,Strawberries,610,8.0
573178,Yuba,Organic Hass Avocado,598,9.0


In [135]:
df_4[df_4.county == "San Bernardino"]

,county,product_name,anzahl,rank
389620,San Bernardino,Original Rice Krispies Treats,6,1.0
389557,San Bernardino,April Fresh Liquid Fabric Softener,4,2.0
389589,San Bernardino,Fabric Softener Dryer Sheet Outdoor Fresh 160C...,4,2.0
389610,San Bernardino,Milk Chocolate Covered Raisins,4,2.0
389615,San Bernardino,Old Fashioned Rods Pretzels,4,2.0
389616,San Bernardino,Organic 2% Reduced Fat Milk,4,2.0
389632,San Bernardino,"Select-A-Size Paper Towels, White, 2 Huge Roll...",4,2.0
389645,San Bernardino,White Antibacterial Deodorant Soap Bars,4,2.0
389647,San Bernardino,With Raspberry Lemonade,4,2.0
389570,San Bernardino,Chocolate Chip Cookie Dough,3,10.0


In [93]:
df_4

,county,product_name,anzahl,rank
233,Alameda,Banana,352,1.0
218,Alameda,Bag of Organic Bananas,269,2.0
654,Alameda,Clementines,177,3.0
2351,Alameda,Organic Strawberries,168,4.0
1219,Alameda,Granny Smith Apples,139,5.0
...,...,...,...,...
576879,Yuba,Sparkling Water Grapefruit,668,6.0
573171,Yuba,Organic Half & Half,653,7.0
577190,Yuba,Strawberries,610,8.0
573178,Yuba,Organic Hass Avocado,598,9.0


In [129]:
df_13 = df_4.groupby("county").agg(coun=("county","count")).reset_index()

In [130]:
df_13[">10"] = df_13.coun > 10

In [106]:
df_13.drop("coun",axis=1,inplace=True)

In [131]:
df_4 = pd.merge(df_4,df_13)

In [132]:
df_4

,county,product_name,anzahl,rank,>10,coun
0,Alameda,Banana,352,1.0,True,11
1,Alameda,Bag of Organic Bananas,269,2.0,True,11
2,Alameda,Clementines,177,3.0,True,11
3,Alameda,Organic Strawberries,168,4.0,True,11
4,Alameda,Granny Smith Apples,139,5.0,True,11
...,...,...,...,...,...,...
574,Yuba,Sparkling Water Grapefruit,668,6.0,False,10
575,Yuba,Organic Half & Half,653,7.0,False,10
576,Yuba,Strawberries,610,8.0,False,10
577,Yuba,Organic Hass Avocado,598,9.0,False,10


In [116]:
fehler = df_4[df_4[">10"] == True].county.unique()

In [117]:
fehler

array(['Alameda', 'Merced', 'Napa', 'San Bernardino', 'Tehama'],
      dtype=object)

In [126]:
df_4[df_4.county == "Alameda"].count().values[0]

11

In [143]:
df_4[df_4.county == "Alameda"].coun.values[0]-10

1

In [147]:
df_4[df_4.county == "Alameda"].tail(df_4[df_4.county == "Alameda"].coun.values[0]-10).index


Int64Index([10], dtype='int64')

In [151]:
for i in fehler:
    df_4.drop(df_4[df_4.county == i].tail(df_4[df_4.county == i].coun.values[0]-10).index,inplace=True)

In [152]:
df_4.head(50)

,county,product_name,anzahl,rank,>10,coun
0,Alameda,Banana,352,1.0,True,11
1,Alameda,Bag of Organic Bananas,269,2.0,True,11
2,Alameda,Clementines,177,3.0,True,11
3,Alameda,Organic Strawberries,168,4.0,True,11
4,Alameda,Granny Smith Apples,139,5.0,True,11
5,Alameda,Sparkling Mineral Water,130,6.0,True,11
6,Alameda,Apples,122,7.0,True,11
7,Alameda,0% Greek Strained Yogurt,120,8.0,True,11
8,Alameda,Soda,106,9.0,True,11
9,Alameda,Organic Granny Smith Apple,102,10.0,True,11


In [8]:
idk = {}
for i in counties:
    idk.update({f"{i}":1})

In [9]:
ableitung = []
test2 = []
for i in idk:
    test = {}
    for j in idk:
        score = 0
        
        for o in df_4.loc[df_4.county == i,("product_name","rank")].values:
            for z in df_4.loc[df_4.county == j,("product_name","rank")].values:
                if o[0] == z[0]:
                    score = score+1
                    if o[1] == z[1]:
                        score = score+1
        if score < 20:
         test.update({f"{j}":score})
         test2.append(i+j)
         if not(i+j in test2 and j+i in test2):
          ableitung.append([score,i,j])

                
    print(score, test)
    idk.update({i:test})
         

7 {'Alpine': 5, 'Amador': 4, 'Calaveras': 5, 'Colusa': 4, 'Contra Costa': 3, 'Del Norte': 3, 'El Dorado': 5, 'Fresno': 5, 'Glenn': 5, 'Humboldt': 2, 'Imperial': 5, 'Inyo': 6, 'Kern': 6, 'Kings': 3, 'Lake': 6, 'Lassen': 3, 'Los Angeles': 3, 'Madera': 6, 'Marin': 3, 'Mariposa': 5, 'Mendocino': 7, 'Merced': 5, 'Modoc': 5, 'Mono': 5, 'Monterey': 6, 'Napa': 8, 'Nevada': 4, 'Orange': 5, 'Placer': 1, 'Plumas': 3, 'Riverside': 5, 'Sacramento': 6, 'San Benito': 5, 'San Bernardino': 0, 'San Diego': 3, 'San Francisco': 6, 'San Joaquin': 3, 'San Luis Obispo': 6, 'San Mateo': 3, 'Santa Barbara': 3, 'Santa Clara': 5, 'Santa Cruz': 5, 'Shasta': 5, 'Sierra': 6, 'Siskiyou': 5, 'Solano': 5, 'Sonoma': 6, 'Stanislaus': 6, 'Sutter': 5, 'Tehama': 2, 'Trinity': 1, 'Tulare': 5, 'Tuolumne': 4, 'Ventura': 5, 'Yolo': 4, 'Yuba': 7}
10 {'Alameda': 5, 'Amador': 7, 'Calaveras': 12, 'Colusa': 10, 'Contra Costa': 8, 'Del Norte': 5, 'El Dorado': 7, 'Fresno': 13, 'Glenn': 12, 'Humboldt': 2, 'Imperial': 10, 'Inyo': 11, '

In [68]:
test2

[{'Alameda', 'Alpine'},
 {'Alameda', 'Amador'},
 {'Alameda', 'Calaveras'},
 {'Alameda', 'Colusa'},
 {'Alameda', 'Contra Costa'},
 {'Alameda', 'Del Norte'},
 {'Alameda', 'El Dorado'},
 {'Alameda', 'Fresno'},
 {'Alameda', 'Glenn'},
 {'Alameda', 'Humboldt'},
 {'Alameda', 'Imperial'},
 {'Alameda', 'Inyo'},
 {'Alameda', 'Kern'},
 {'Alameda', 'Kings'},
 {'Alameda', 'Lake'},
 {'Alameda', 'Lassen'},
 {'Alameda', 'Los Angeles'},
 {'Alameda', 'Madera'},
 {'Alameda', 'Marin'},
 {'Alameda', 'Mariposa'},
 {'Alameda', 'Mendocino'},
 {'Alameda', 'Merced'},
 {'Alameda', 'Modoc'},
 {'Alameda', 'Mono'},
 {'Alameda', 'Monterey'},
 {'Alameda', 'Napa'},
 {'Alameda', 'Nevada'},
 {'Alameda', 'Orange'},
 {'Alameda', 'Placer'},
 {'Alameda', 'Plumas'},
 {'Alameda', 'Riverside'},
 {'Alameda', 'Sacramento'},
 {'Alameda', 'San Benito'},
 {'Alameda', 'San Bernardino'},
 {'Alameda', 'San Diego'},
 {'Alameda', 'San Francisco'},
 {'Alameda', 'San Joaquin'},
 {'Alameda', 'San Luis Obispo'},
 {'Alameda', 'San Mateo'},
 

In [36]:
idk["Alameda"]

{'Alpine': 5,
 'Amador': 4,
 'Calaveras': 5,
 'Colusa': 4,
 'Contra Costa': 3,
 'Del Norte': 3,
 'El Dorado': 5,
 'Fresno': 5,
 'Glenn': 5,
 'Humboldt': 2,
 'Imperial': 5,
 'Inyo': 6,
 'Kern': 6,
 'Kings': 3,
 'Lake': 6,
 'Lassen': 3,
 'Los Angeles': 3,
 'Madera': 6,
 'Marin': 3,
 'Mariposa': 5,
 'Mendocino': 7,
 'Merced': 5,
 'Modoc': 5,
 'Mono': 5,
 'Monterey': 6,
 'Napa': 8,
 'Nevada': 4,
 'Orange': 5,
 'Placer': 1,
 'Plumas': 3,
 'Riverside': 5,
 'Sacramento': 6,
 'San Benito': 5,
 'San Bernardino': 0,
 'San Diego': 3,
 'San Francisco': 6,
 'San Joaquin': 3,
 'San Luis Obispo': 6,
 'San Mateo': 3,
 'Santa Barbara': 3,
 'Santa Clara': 5,
 'Santa Cruz': 5,
 'Shasta': 5,
 'Sierra': 6,
 'Siskiyou': 5,
 'Solano': 5,
 'Sonoma': 6,
 'Stanislaus': 6,
 'Sutter': 5,
 'Tehama': 2,
 'Trinity': 1,
 'Tulare': 5,
 'Tuolumne': 4,
 'Ventura': 5,
 'Yolo': 4,
 'Yuba': 7}

In [108]:
liste = ["AlamedaAlpine"]

In [81]:
for i in liste:
    dup = {aa,bb}
    print(i.type)

AttributeError: 'set' object has no attribute 'type'

In [10]:
ableitung.sort(reverse=True)

In [68]:
df_4[df_4.county == "Alameda"]

,county,product_name,anzahl,rank
233,Alameda,Banana,352,1.0
218,Alameda,Bag of Organic Bananas,269,2.0
654,Alameda,Clementines,177,3.0
2351,Alameda,Organic Strawberries,168,4.0
1219,Alameda,Granny Smith Apples,139,5.0
3083,Alameda,Sparkling Mineral Water,130,6.0
165,Alameda,Apples,122,7.0
1,Alameda,0% Greek Strained Yogurt,120,8.0
3047,Alameda,Soda,106,9.0
2106,Alameda,Organic Granny Smith Apple,102,10.0


In [148]:
df_4[df_4.county == "Orange"]

,county,product_name,anzahl,rank
330422,Orange,Banana,6030,1.0
330318,Orange,Bag of Organic Bananas,3654,2.0
342582,Orange,Organic Strawberries,3012,3.0
340759,Orange,Organic Baby Spinach,2792,4.0
341625,Orange,Organic Hass Avocado,2495,5.0
340738,Orange,Organic Avocado,2439,6.0
338131,Orange,Large Lemon,2001,7.0
338585,Orange,Limes,1981,8.0
342908,Orange,Organic Whole Milk,1967,9.0
342240,Orange,Organic Raspberries,1755,10.0


In [149]:
df_4[df_4.county == "Calaveras"]

,county,product_name,anzahl,rank
31399,Calaveras,Banana,7381,1.0
31281,Calaveras,Bag of Organic Bananas,5192,2.0
45259,Calaveras,Organic Strawberries,4177,3.0
43256,Calaveras,Organic Baby Spinach,3839,4.0
44197,Calaveras,Organic Hass Avocado,3542,5.0
43233,Calaveras,Organic Avocado,2937,6.0
40238,Calaveras,Large Lemon,2312,7.0
40737,Calaveras,Limes,2251,8.0
50558,Calaveras,Strawberries,2140,9.0
44890,Calaveras,Organic Raspberries,2137,10.0


In [67]:
df_4[df_4.county == "San Bernardino"]

,county,product_name,anzahl,rank
389620,San Bernardino,Original Rice Krispies Treats,6,1.0
389557,San Bernardino,April Fresh Liquid Fabric Softener,4,2.0
389589,San Bernardino,Fabric Softener Dryer Sheet Outdoor Fresh 160C...,4,2.0
389610,San Bernardino,Milk Chocolate Covered Raisins,4,2.0
389615,San Bernardino,Old Fashioned Rods Pretzels,4,2.0
389616,San Bernardino,Organic 2% Reduced Fat Milk,4,2.0
389632,San Bernardino,"Select-A-Size Paper Towels, White, 2 Huge Roll...",4,2.0
389645,San Bernardino,White Antibacterial Deodorant Soap Bars,4,2.0
389647,San Bernardino,With Raspberry Lemonade,4,2.0
389570,San Bernardino,Chocolate Chip Cookie Dough,3,10.0


maybe gruppen bilden oder über alle

In [13]:
ableitung

[[18, 'Imperial', 'Santa Clara'],
 [18, 'Calaveras', 'Orange'],
 [18, 'Calaveras', 'Glenn'],
 [17, 'Lake', 'Monterey'],
 [17, 'Kern', 'Monterey'],
 [17, 'Glenn', 'Sutter'],
 [17, 'Fresno', 'Orange'],
 [16, 'Santa Clara', 'Siskiyou'],
 [16, 'Orange', 'Sutter'],
 [16, 'Mariposa', 'Santa Cruz'],
 [16, 'Mariposa', 'Mono'],
 [16, 'Glenn', 'Orange'],
 [16, 'Calaveras', 'Sutter'],
 [16, 'Calaveras', 'Siskiyou'],
 [16, 'Calaveras', 'Plumas'],
 [16, 'Calaveras', 'Monterey'],
 [16, 'Calaveras', 'Kern'],
 [16, 'Calaveras', 'Fresno'],
 [15, 'Siskiyou', 'Ventura'],
 [15, 'Siskiyou', 'Sutter'],
 [15, 'Orange', 'Siskiyou'],
 [15, 'Orange', 'Santa Cruz'],
 [15, 'Monterey', 'Stanislaus'],
 [15, 'Monterey', 'Orange'],
 [15, 'Mono', 'Santa Cruz'],
 [15, 'Mariposa', 'Orange'],
 [15, 'Madera', 'San Francisco'],
 [15, 'Madera', 'Monterey'],
 [15, 'Madera', 'Mendocino'],
 [15, 'Kern', 'Stanislaus'],
 [15, 'Kern', 'Orange'],
 [15, 'Imperial', 'Ventura'],
 [15, 'Glenn', 'Siskiyou'],
 [15, 'Glenn', 'San Benito'

In [45]:
for i in ableitung:
    print(i)

[18, 'Santa Clara', 'Imperial']
[18, 'Orange', 'Calaveras']
[18, 'Imperial', 'Santa Clara']
[18, 'Glenn', 'Calaveras']
[18, 'Calaveras', 'Orange']
[18, 'Calaveras', 'Glenn']
[17, 'Sutter', 'Glenn']
[17, 'Orange', 'Fresno']
[17, 'Monterey', 'Lake']
[17, 'Monterey', 'Kern']
[17, 'Lake', 'Monterey']
[17, 'Kern', 'Monterey']
[17, 'Glenn', 'Sutter']
[17, 'Fresno', 'Orange']
[16, 'Sutter', 'Orange']
[16, 'Sutter', 'Calaveras']
[16, 'Siskiyou', 'Santa Clara']
[16, 'Siskiyou', 'Calaveras']
[16, 'Santa Cruz', 'Mariposa']
[16, 'Santa Clara', 'Siskiyou']
[16, 'Plumas', 'Calaveras']
[16, 'Orange', 'Sutter']
[16, 'Orange', 'Glenn']
[16, 'Monterey', 'Calaveras']
[16, 'Mono', 'Mariposa']
[16, 'Mariposa', 'Santa Cruz']
[16, 'Mariposa', 'Mono']
[16, 'Kern', 'Calaveras']
[16, 'Glenn', 'Orange']
[16, 'Fresno', 'Calaveras']
[16, 'Calaveras', 'Sutter']
[16, 'Calaveras', 'Siskiyou']
[16, 'Calaveras', 'Plumas']
[16, 'Calaveras', 'Monterey']
[16, 'Calaveras', 'Kern']
[16, 'Calaveras', 'Fresno']
[15, 'Ventura'

In [29]:
final = [[2,"set"],[2,"aaa"],[5,"da"],[6,"lambda"]]

In [30]:
final.sort()

In [31]:
final

[[2, 'aaa'], [2, 'set'], [5, 'da'], [6, 'lambda']]

In [14]:
df_4

,county,product_name,anzahl,rank
233,Alameda,Banana,352,1.0
218,Alameda,Bag of Organic Bananas,269,2.0
654,Alameda,Clementines,177,3.0
2351,Alameda,Organic Strawberries,168,4.0
1219,Alameda,Granny Smith Apples,139,5.0
...,...,...,...,...
576879,Yuba,Sparkling Water Grapefruit,668,6.0
573171,Yuba,Organic Half & Half,653,7.0
577190,Yuba,Strawberries,610,8.0
573178,Yuba,Organic Hass Avocado,598,9.0


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [14]:
ableitung = []
test2 = []
for i in idk:
    test = {}
    for j in idk:
        score = 0
        score = cosine_similarity(df_8[df_8.county == i].drop("county",axis=1).values,df_8[df_8.county == j].drop("county",axis=1).values).diagonal().sum()
     
        if score < 10:
        #  test.update({f"{j}":score})
         test2.append(i+j)
         if not(i+j in test2 and j+i in test2):
          ableitung.append([score,i,j])

                
    #print(score, test)
    #idk.update({i:test})

In [63]:
ableitung.sort(reverse=True)

In [15]:
ableitung

[[5.999999999999998, 'Alameda', 'Alpine'],
 [4.999999999999999, 'Alameda', 'Amador'],
 [5.999999999999998, 'Alameda', 'Calaveras'],
 [4.499999999999999, 'Alameda', 'Colusa'],
 [4.999999999999999, 'Alameda', 'Contra Costa'],
 [4.999999999999999, 'Alameda', 'Del Norte'],
 [5.499999999999998, 'Alameda', 'El Dorado'],
 [5.999999999999998, 'Alameda', 'Fresno'],
 [5.999999999999998, 'Alameda', 'Glenn'],
 [4.499999999999999, 'Alameda', 'Humboldt'],
 [5.999999999999998, 'Alameda', 'Imperial'],
 [6.499999999999998, 'Alameda', 'Inyo'],
 [6.499999999999998, 'Alameda', 'Kern'],
 [4.999999999999999, 'Alameda', 'Kings'],
 [6.499999999999998, 'Alameda', 'Lake'],
 [5.499999999999998, 'Alameda', 'Lassen'],
 [4.999999999999999, 'Alameda', 'Los Angeles'],
 [6.499999999999998, 'Alameda', 'Madera'],
 [4.999999999999999, 'Alameda', 'Marin'],
 [5.999999999999998, 'Alameda', 'Mariposa'],
 [6.499999999999998, 'Alameda', 'Mendocino'],
 [6.499999999999998, 'Alameda', 'Merced'],
 [5.999999999999998, 'Alameda', 'M

In [66]:
df_4[df_4.county == "Mariposa"]

,county,product_name,anzahl,rank
239904,Mariposa,Banana,3466,1.0
239805,Mariposa,Bag of Organic Bananas,2405,2.0
251037,Mariposa,Organic Strawberries,1746,3.0
249417,Mariposa,Organic Baby Spinach,1592,4.0
250197,Mariposa,Organic Hass Avocado,1372,5.0
255363,Mariposa,Strawberries,1256,6.0
249399,Mariposa,Organic Avocado,1174,7.0
246987,Mariposa,Large Lemon,1129,8.0
251332,Mariposa,Organic Whole Milk,1016,9.0
247388,Mariposa,Limes,1010,10.0


In [65]:
df_4[df_4.county == "Mono"]

,county,product_name,anzahl,rank
292269,Mono,Banana,502,1.0
292249,Mono,Bag of Organic Bananas,484,2.0
294973,Mono,Organic Strawberries,335,3.0
294182,Mono,Organic Baby Spinach,274,4.0
294544,Mono,Organic Hass Avocado,243,5.0
295936,Mono,Strawberries,181,6.0
294822,Mono,Organic Raspberries,166,7.0
293661,Mono,Large Lemon,159,8.0
295112,Mono,Organic Whole Milk,158,9.0
294629,Mono,Organic Lemon,156,10.0


In [13]:
df_7 = df_4[["county","product_name","rank"]].copy()
df_8 = pd.get_dummies(df_7,columns=["product_name","rank"])

In [19]:
df_8

,county,product_name_0% Greek Strained Yogurt,product_name_100% Prune Juice,product_name_100% Raw Coconut Water,product_name_100% Whole Wheat Bread,product_name_2% Reduced Fat Milk,product_name_All American Salad,product_name_Almond Breeze Original Almond Milk,product_name_Apple Honeycrisp Organic,product_name_Apples,...,rank_1.0,rank_2.0,rank_3.0,rank_4.0,rank_5.0,rank_6.0,rank_7.0,rank_8.0,rank_9.0,rank_10.0
233,Alameda,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
218,Alameda,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
654,Alameda,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2351,Alameda,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1219,Alameda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576879,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
573171,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
577190,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
573178,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
Alameda = df_8[df_8.county == "Alameda"].drop("county",axis=1)

In [63]:
San_Bernardino = df_8[df_8.county == "San Bernardino"].drop("county",axis=1)


In [21]:
Alameda

,product_name_0% Greek Strained Yogurt,product_name_100% Prune Juice,product_name_100% Raw Coconut Water,product_name_100% Whole Wheat Bread,product_name_2% Reduced Fat Milk,product_name_All American Salad,product_name_Almond Breeze Original Almond Milk,product_name_Apple Honeycrisp Organic,product_name_Apples,product_name_April Fresh Liquid Fabric Softener,...,rank_1.0,rank_2.0,rank_3.0,rank_4.0,rank_5.0,rank_6.0,rank_7.0,rank_8.0,rank_9.0,rank_10.0
233,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
218,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
654,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
165,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [29]:
#Alameda = sparse.csr_matrix(Alameda)

In [35]:
Monterey = df_8[df_8.county == "Monterey"].drop("county",axis=1)
#Calaveras = sparse.csr_matrix(Calaveras)

In [40]:
Marin = df_8[df_8.county == "Marin"].drop("county",axis=1)

In [23]:
Glenn.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [21]:
Calaveras.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [36]:
pd.concat([Calaveras,Monterey])

,product_name_0% Greek Strained Yogurt,product_name_100% Prune Juice,product_name_100% Raw Coconut Water,product_name_100% Whole Wheat Bread,product_name_2% Reduced Fat Milk,product_name_All American Salad,product_name_Almond Breeze Original Almond Milk,product_name_Apple Honeycrisp Organic,product_name_Apples,product_name_April Fresh Liquid Fabric Softener,...,rank_1.0,rank_2.0,rank_3.0,rank_4.0,rank_5.0,rank_6.0,rank_7.0,rank_8.0,rank_9.0,rank_10.0
31399,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
31281,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
45259,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
43256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
44197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
43233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
40238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
40737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
50558,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [46]:
Alameda.drop(Alameda.tail(1).index,inplace=True)

In [69]:
Alameda = sparse.csr_matrix(Alameda.values)
Orange = sparse.csr_matrix(Orange.values)
#Alameda.values


AttributeError: values not found

In [41]:
Alameda.values*Calaveras.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [41]:
df_10 = cosine_similarity(Alameda,pd.concat([Marin,Monterey]))

In [43]:
df_10.sum()

13.999999999999996

In [56]:
cosine_similarity(Alameda,Marin)

array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5]])

In [64]:
u, s, vh = np.linalg.svd(cosine_similarity(Alameda,San_Bernardino))

In [65]:
X2D = cosine_similarity(Alameda,San_Bernardino) @ vh[1,:].T

In [66]:
X2D.sum()

1.414213562373095

In [75]:
ableitung = []
test2 = []
for i in idk:
    test = {}
    for j in idk:
         for k in idk:   
                score = 0
            
                for o in df_4.loc[df_4.county == i,("product_name","rank")].values:
                    for z in df_4.loc[df_4.county == j,("product_name","rank")].values:
                        for l in df_4.loc[df_4.county == k,("product_name","rank")].values:
                            if o[0] == z[0] == l[0]:
                                score = score+1
                                if o[1] == z[1] == l[1]:
                                    score = score+1
                if score < 30:
                    #test.update({f"{j}":score})
                    test2.append(i+j)
                    if not(i+j+k in test2 and j+i+k in test2):
                     ableitung.append([score,i,j])

                
    # print(score, test)
    # idk.update({i:test})

KeyboardInterrupt: 

In [ ]:
ableitung = []
test2 = []
for i in idk:
    test = {}
    for j in idk:
        score = 0
        score = cosine_similarity(df_8[df_8.county == i].drop("county",axis=1).values,df_8[df_8.county == j].drop("county",axis=1).values).diagonal().sum()
     
        if score < 10:
        #  test.update({f"{j}":score})
         test2.append(i+j)
         if not(i+j in test2 and j+i in test2):
          ableitung.append([score,i,j])

                
    #print(score, test)
    #idk.update({i:test})

In [78]:
ableitung

[[5, 'Alameda', 'Alpine'],
 [4, 'Alameda', 'Amador'],
 [5, 'Alameda', 'Calaveras'],
 [4, 'Alameda', 'Colusa'],
 [3, 'Alameda', 'Contra Costa'],
 [3, 'Alameda', 'Del Norte'],
 [5, 'Alameda', 'El Dorado'],
 [5, 'Alameda', 'Fresno'],
 [5, 'Alameda', 'Glenn'],
 [2, 'Alameda', 'Humboldt'],
 [5, 'Alameda', 'Imperial'],
 [6, 'Alameda', 'Inyo'],
 [6, 'Alameda', 'Kern'],
 [3, 'Alameda', 'Kings'],
 [6, 'Alameda', 'Lake'],
 [3, 'Alameda', 'Lassen'],
 [3, 'Alameda', 'Los Angeles'],
 [6, 'Alameda', 'Madera'],
 [3, 'Alameda', 'Marin'],
 [5, 'Alameda', 'Mariposa'],
 [7, 'Alameda', 'Mendocino'],
 [5, 'Alameda', 'Merced'],
 [5, 'Alameda', 'Modoc'],
 [5, 'Alameda', 'Mono'],
 [6, 'Alameda', 'Monterey'],
 [8, 'Alameda', 'Napa'],
 [4, 'Alameda', 'Nevada'],
 [5, 'Alameda', 'Orange'],
 [1, 'Alameda', 'Placer'],
 [3, 'Alameda', 'Plumas'],
 [5, 'Alameda', 'Riverside'],
 [6, 'Alameda', 'Sacramento'],
 [5, 'Alameda', 'San Benito'],
 [0, 'Alameda', 'San Bernardino'],
 [3, 'Alameda', 'San Diego'],
 [6, 'Alameda', 

In [81]:
df_12= df.groupby("county").agg(order=("order_id","nunique")).reset_index()

In [93]:
df_12.order.median()

7548.0

In [101]:
liste1 = df_12.loc[df_12.order > df_12.order.median()].county.to_list()

In [103]:
liste2 = df_12.loc[df_12.order <= df_12.order.median()].county.to_list()

In [107]:
liste1.__len__()

28

In [113]:
liste1_score = 0
liste2_score = 0
len1 = 0
len2 = 0
for i in ableitung:
    if (i[1] in liste1 and i[2] in liste1):
        liste1_score = liste1_score + i[0]
        len1 = len1 + 1
    elif (i[1] in liste2 and i[2] in liste2):
        liste2_score = liste2_score + i[0]
        len2 = len2 + 1
liste1_score = liste1_score/len1
liste2_score = liste2_score/len2

In [114]:
liste1_score

10.82010582010582

In [115]:
liste2_score

5.532019704433497